In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
def detection(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

# Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    person_boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
            # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
            # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            if label == 'person':
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                person_boxes.append((x, y, w, h))  
    return (img, person_boxes)

In [3]:
# Tracking type
from __future__ import print_function
import sys
import cv2
from random import randint

trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
    return tracker

In [4]:
# Task 1a
# Load Yolo
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
classes = []
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
directory = 'Group_Component/sequence'
detected_img = []

img_count = 0
pedestrain_decrease = 0
for images in os.listdir(directory):
    img = cv2.imread(os.path.join(directory,images))
    temp = detection(img)
    detected_img.append((images,temp))
    #plt.imsave(fname = 'D:/Master_learning/9517/Project/Group/test/'+images, arr = temp[0])
    # cv2.imwrite('output1b/'+images, temp[0])

In [165]:
# print(len(detected_img))
# print(detected_img[2])
for i in range(len(detected_img[12][1][1])):
    print(len(detected_img[12][1][1][i]))
    x, y, w, h = detected_img[12][1][1][i]
    print(x, y, w, h)
    

4
451 164 35 81
4
324 201 34 91
4
529 237 34 87
4
734 293 33 116


In [5]:
# Task 1b
# Multi_tracking
# Specify the tracker type
trackerType = "CSRT"

# Create MultiTracker object
multiTracker = cv2.MultiTracker_create()


points_list = []
point_size = 1
point_color = (0, 0, 255) # BGR
thickness = 4


# Detecting from the first frame
# detected_img[0] -> the first image in the data set
# detected_img[0][1] -> the tuple: (temp[0], temp[1]) -> (img, boxes)
# detected_img[0][1][1] -> the list of all bounding boxes in the first image
for i in range(len(detected_img[0][1][1])):
    x, y, w, h = detected_img[0][1][1][i]
    bbox = (x, y, w, h)
    # Tracking point
    points_list.append([(x + w//2, y + h//2)])
    # Initialize MultiTracker
    multiTracker.add(createTrackerByName(trackerType), detected_img[0][1][0], bbox)

for j in range(1, len(detected_img)):
    # get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(detected_img[j][1][0])   # detected_img[i][1][0] == temp[0]
    # draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        points_list[i].append((int(newbox[0] + newbox[2]/2), int(newbox[1] + newbox[3]/2)))
        cv2.rectangle(detected_img[j][1][0], p1, p2, colors[i], 2, 1)
        cv2.putText(detected_img[j][1][0],str(i),p1,cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
        for points in points_list:
            for point in points:
                cv2.circle(detected_img[j][1][0], point, point_size, point_color, thickness)

    # show frame
    cv2.imwrite('output1b/'+detected_img[j][0], detected_img[j][1][0])